In [2]:
import numpy as np
import pandas as pd
from cmapPy.pandasGEXpress.parse import parse
import cmapPy.pandasGEXpress.write_gct as wg
import cmapPy.pandasGEXpress.write_gctx as wgx

gctx_file =      "GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx" # 20 gigs 😳
sig_info_file =  "GSE92742_Broad_LINCS_sig_info.txt"
gene_info_file = "GSE92742_Broad_LINCS_gene_info.txt"
pert_info_file = "GSE92742_Broad_LINCS_pert_info.txt"
cell_info_file = "GSE92742_Broad_LINCS_cell_info.txt"

In [3]:
# get metadata

# columns
sig_info = pd.read_csv(
    sig_info_file,
    sep="\t"
)
sig_info.set_index("sig_id", inplace=True)

# rows
gene_info = pd.read_csv(
    gene_info_file,
    sep="\t",
    dtype={'pr_gene_id': 'str'}
)
gene_info.set_index("pr_gene_id", inplace=True)

/home/jops/miniconda3/envs/my_cmapPy_env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# get perturbagen metadata
pert_info = pd.read_csv(
    pert_info_file,
    sep="\t",
    na_values=["-666", -666],
    index_col="pert_id"
)
pert_info['pubchem_cid'] = pd.to_numeric(pert_info['pubchem_cid'], errors='coerce').fillna('0').astype('Int64')

In [5]:
# get cell line metadata
cell_info = pd.read_csv(
    cell_info_file,
    sep="\t",
    na_values=["-666", -666],
    index_col="cell_id"
)

In [94]:
mouse_genes = pd.read_csv('ncbi_gene/Mus_musculus.gene_info', sep='\t')
human_genes = pd.read_csv('ncbi_gene/Homo_sapiens.gene_info', sep='\t')
orthologs = pd.read_csv('ncbi_gene/gene_orthologs', sep='\t')

In [69]:
orthologs.rename(columns={"#tax_id": "tax_id"}, inplace=True)
mouse_tax_id = 10090
human_tax_id = 9606
mouse_human_ortho = (orthologs['tax_id'] == human_tax_id) & (orthologs['Other_tax_id'] == mouse_tax_id)
human_mouse_ortho = (orthologs['tax_id'] == mouse_tax_id) & (orthologs['Other_tax_id'] == human_tax_id)
orthologs = orthologs[mouse_human_ortho | human_mouse_ortho]

In [134]:
def mouse_to_human(sym):
    mouse_id = mouse_genes[mouse_genes['Symbol'] == sym]['GeneID'].values
    human_id = pd.concat([
        orthologs[orthologs['GeneID'].isin(mouse_id)]['Other_GeneID'],
        orthologs[orthologs['Other_GeneID'].isin(mouse_id)]['GeneID']
    ]).values
    try:
        return human_genes[human_genes['GeneID'].isin(human_id)]['Symbol'].values[0]
    except IndexError:
        return

In [188]:
# get genes from Ficek paper
ficek = pd.read_csv(
    "paper_ficek.csv"
)
ficek = pd.concat([ficek['Gene'].rename('Mouse'), ficek['Gene'].apply(mouse_to_human).rename('Human')], axis=1)

In [189]:
# get genes from Miller paper
miller = pd.read_csv('paper_miller.csv')
miller = pd.concat([miller['Symbol'].rename('Mouse'), miller['Symbol'].apply(mouse_to_human).rename('Human')], axis=1)

In [190]:
# get genes from Bagot paper
bagot = pd.read_csv('paper_bagot.csv')
bagot = pd.concat([bagot['Symbol'].rename('Mouse'), bagot['Symbol'].apply(mouse_to_human).rename('Human')], axis=1)

In [191]:
# write to csv so corrections can be made manually
pd.concat([ficek, miller, bagot], ignore_index=True).to_csv('paper_all.csv')

In [213]:
potential_genes = pd.read_csv('paper_all_.csv', index_col=0)
potential_genes = potential_genes[potential_genes['Human'].notnull()]['Human']
potential_genes

0        ARRDC2
1         PLIN4
2        SLC2A1
3       TSC22D3
4         HIF3A
5        MAP3K6
6         TXNIP
7          SGK1
8        NFKBIA
9       PLEKHF1
10         RHOJ
11       CSRNP1
12         RGCC
13       MFSD2A
14      DCLRE1B
15        FGF11
16        ITGAD
17      SLC27A3
18        DDIT4
19      GADD45G
20       POLR3E
21      PPP1R3G
22      SULT1A1
23       MS4A15
24         AQP1
25        EOMES
26       DOC2GP
27        VIPR2
28        FRMD7
29          NMB
         ...   
90        MFAP2
91        DNAH9
92      TSPAN11
93        FGF23
94         AGRP
95     DNASE1L1
96         CD70
97       TUBAL3
98          DCT
99      LGALS12
100      AXDND1
101     FAM209A
102       TDGF1
103      ZSWIM2
104      TM6SF2
105      CCDC38
106        FUT7
107      HAVCR1
108      CX3CR1
109        HELT
110        GJA8
113        NXF3
114       TRIM5
115     KIR3DL2
117      SLFN13
119       VPS25
128      TEX261
129     CEP57L1
130       MRNIP
131       TTC7A
Name: Human, Length: 118

In [205]:
norketamine = parse("norketamine.gct", make_multiindex=True)

In [226]:
criterion = norketamine.row_metadata_df['pr_gene_symbol'].isin(potential_genes)
df = pd.concat([norketamine.row_metadata_df, norketamine.data_df.iloc[:, 0].rename('z_score')], axis=1)
df['z_score_abs'] = df['z_score'].abs()
df = df.sort_values(by=['z_score_abs'], ascending=False).drop('z_score_abs', 1)
#df[(df['z_score'] > 0.25) | (df['z_score'] < -0.25)]['pr_gene_symbol'].to_csv('norketamine_deg_0.25.txt', index=False)
df[(df['z_score'] > 2) | (df['z_score'] < -2)]

,pr_gene_symbol,pr_gene_title,pr_is_lm,pr_is_bing,z_score
rid,,,,,
1050,CEBPA,CCAAT/enhancer binding protein alpha,1,1,7.0766
10954,PDIA5,protein disulfide isomerase family A member 5,1,1,7.0157
23131,GPATCH8,G-patch domain containing 8,1,1,6.8505
55699,IARS2,"isoleucyl-tRNA synthetase 2, mitochondrial",1,1,-6.8265
2065,ERBB3,erb-b2 receptor tyrosine kinase 3,1,1,6.7713
64943,NT5DC2,5'-nucleotidase domain containing 2,1,1,-6.4348
23512,SUZ12,SUZ12 polycomb repressive complex 2 subunit,1,1,-6.4246
1958,EGR1,early growth response 1,1,1,6.0327
5921,RASA1,RAS p21 protein activator 1,1,1,5.8771


In [222]:
# genes from papers but not in L1000
set(potential_genes.values) - set(df['pr_gene_symbol'].values)

{'ARRDC2',
 'AXDND1',
 'C2orf50',
 'CCDC171',
 'CCDC38',
 'CEP57L1',
 'CFAP100',
 'CFAP206',
 'CSRNP1',
 'DNAAF4',
 'DOC2GP',
 'EFCAB5',
 'EOMES',
 'FAM209A',
 'FGF11',
 'FNDC1',
 'FRMD7',
 'HELT',
 'HTR5BP',
 'INCA1',
 'ITGAD',
 'KIR3DL2',
 'LGALS12',
 'LGR6',
 'LY6G6E',
 'MFSD2A',
 'MRNIP',
 'MS4A15',
 'NPFFR2',
 'PLIN4',
 'PPP1R3G',
 'PYROXD2',
 'RAB37',
 'RHOJ',
 'RSPO1',
 'SHISA8',
 'SLFN13',
 'SP7',
 'SP8',
 'TCEANC',
 'TDGF1',
 'TFAP2D',
 'TM6SF2',
 'TSPAN11',
 'TTC7A',
 'VPS25',
 'WDFY4',
 'ZNF618',
 'ZSWIM2'}